# Initial Setting

In [1]:
import nest_asyncio
nest_asyncio.apply()

import numpy as np
import tensorflow as tf
import random

FRACTION=0.02
BATCH_SIZE = 10 # inf = -1
NUM_EPOCHS = 5 # fixed!
TRAINING_ROUNDS=2

CLIENTS_SHUFFLE_PER_ROUND=False
#CLIENTS_SHUFFLE_PER_ROUND=True 

In [2]:
import os
import time
import sys
import csv

class ParameterSaver:
    def __init__(self):
        self.save_path = os.getcwd()
        
        now = time.localtime()
        self.directory_name = "parameter_set_"+"%04d%02d%02d%02d%02d%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
        
        os.mkdir(os.path.join(self.save_path, self.directory_name))
        print(f"{self.directory_name} directory is created in {self.save_path}")
        
        self.current_round_directory_name=""

    def save_initial_parameter(self, initial_parameter) :
        np.savetxt(os.path.join(self.save_path, self.directory_name, "initial_parameter.csv"), initial_parameter, fmt='%s', delimiter=',')
        
    def round_start(self, round_number):
        self.current_round_directory_name=f"round_{round_number:06d}"
        os.mkdir(os.path.join(self.save_path, self.directory_name, self.current_round_directory_name))
        
    def save_local_parameter(self, client_id, local_parameter) :
        np.savetxt(os.path.join(self.save_path, self.directory_name, self.current_round_directory_name, f"local_parameter_cli_{client_id}.csv"), local_parameter, fmt='%s', delimiter=',')
        
    def save_aggreated_global_parameter(self, aggregated_global_parameter) :
        np.savetxt(os.path.join(self.save_path, self.directory_name, self.current_round_directory_name, f"aggregated_global_parameter.csv"), aggregated_global_parameter, fmt='%s', delimiter=',')

        
#---------------------------------------------------------------------------------
parameter_saver= ParameterSaver() # make directory for saving parameter set

parameter_set_20210219180301 directory is created in D:\jupyter\fed_network_framework\FL_in_one_script


# Make Preprocessed-(I.I.D)Dataset

In [3]:
mnist_train, mnist_test = tf.keras.datasets.mnist.load_data() # This dataset is not "E"mnist. Don't confuse!

raw_dataset_for_iid=list(zip(mnist_train[0].reshape(-1, 28, 28, 1).astype("float32")/255.0, mnist_train[1].astype("float32")))
random.shuffle(raw_dataset_for_iid)

el_size=600
temp_list_for_image=[]
temp_list_for_label=[]
federated_train_data_for_iid=[]
for idx, el in enumerate(raw_dataset_for_iid) :
    temp_list_for_image.append(el[0])
    temp_list_for_label.append(el[1])
    if (idx+1)%(el_size)==0 :
        federated_train_data_for_iid.append((np.array(temp_list_for_image, dtype="float32"), np.array(temp_list_for_label, dtype="float32")))
        temp_list_for_image=[]
        temp_list_for_label=[]
        
federated_train_data = federated_train_data_for_iid

# Make MNIST-CNN 99% model using Keras

In [4]:
keras_model= tf.keras.models.Sequential([
    tf.keras.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, kernel_size=(5, 5), activation="relu", padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    
    tf.keras.layers.Conv2D(64, kernel_size=(5, 5), activation="relu", padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation="softmax"),
])

keras_model.summary()

keras_model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1606144   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5

# Start Training

In [5]:
TOTAL_CLIENTS = len(federated_train_data)
SELECTED_CLIENTS = int(TOTAL_CLIENTS*FRACTION)
print("total client :", TOTAL_CLIENTS, ", selected client :", SELECTED_CLIENTS)

# starting to training
selected_clients_list=clients_status_list=np.random.choice(TOTAL_CLIENTS, size=SELECTED_CLIENTS, replace=False) # that is relevant to 4-2 step.

global_parameter=keras_model.get_weights()
parameter_saver.save_initial_parameter(global_parameter)

print("-- prameter shape --")
for layer in global_parameter :
    print(layer.shape)

list_of_local_parameter=[]
list_of_local_dataset_size=[]
list_of_local_accuracy=[]
list_of_local_loss=[]

for round in range(TRAINING_ROUNDS) :
    print("\n▶ Round", round+1, "◀")
    parameter_saver.round_start(round+1)
    
        # check whether to apply shuffle mode per round
    if CLIENTS_SHUFFLE_PER_ROUND == True :
        selected_clients_list = np.random.choice(TOTAL_CLIENTS, size=SELECTED_CLIENTS, replace=False)
    print("selected clients :", selected_clients_list)

        # recevie Local parameter.
    for client_dataset in selected_clients_list :
        train_images, train_labels=federated_train_data[client_dataset]
        
        keras_model.set_weights(global_parameter)
        
        train_result=keras_model.fit(train_images, train_labels, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=0)
            
        local_parameter=keras_model.get_weights()
        list_of_local_parameter.append(local_parameter)
        parameter_saver.save_local_parameter(client_dataset, local_parameter)
        list_of_local_dataset_size.append(len(train_images))
        list_of_local_accuracy.append(train_result.history["accuracy"][-1])
        list_of_local_loss.append(train_result.history["loss"][-1])
        
        print("    clint ID :", client_dataset, "training complete.")
        print("        accuracy :", train_result.history["accuracy"][-1], "- loss :", train_result.history["loss"][-1])
    
        #4-5. aggregate Local parameters.
    global_parameter = np.mean(list_of_local_parameter, axis=0)
    #global_parameter = np.mean(list_of_local_parameter, axis=0)*np.sum(list_of_local_dataset_size)
    #print("global_parameter :",global_parameter)
    parameter_saver.save_aggreated_global_parameter(global_parameter)
    current_mean_accuracy = np.mean(np.array(list_of_local_accuracy, dtype="float32"))
    current_mean_loss = np.mean(np.array(list_of_local_loss, dtype="float32"))
    print(f"  evaluation mean : accuracy - {current_mean_accuracy}, loss - {current_mean_loss}")   
    
    list_of_local_parameter.clear()
    list_of_local_dataset_size.clear()
    list_of_local_accuracy.clear()
    list_of_local_loss.clear()
    
print("\n\n▶▶▶ Round is over.")

total client : 100 , selected client : 2
-- prameter shape --
(5, 5, 1, 32)
(32,)
(5, 5, 32, 64)
(64,)
(3136, 512)
(512,)
(512, 10)
(10,)

▶ Round 1 ◀
selected clients : [62  9]
    clint ID : 62 training complete.
        accuracy : 0.7599999904632568 - loss : 1.7014524936676025
    clint ID : 9 training complete.
        accuracy : 0.9233333468437195 - loss : 1.548226237297058
  evaluation mean : accuracy - 0.8416666984558105, loss - 1.6248393058776855

▶ Round 2 ◀
selected clients : [62  9]
    clint ID : 62 training complete.
        accuracy : 0.9733333587646484 - loss : 1.489459753036499
    clint ID : 9 training complete.
        accuracy : 0.9616666436195374 - loss : 1.502359390258789
  evaluation mean : accuracy - 0.9674999713897705, loss - 1.495909571647644


▶▶▶ Round is over.
